In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    
load_dotenv()

True

In [2]:
# load datasets
from utils.utils import date_range, preprocess_c

df_train = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 1), 
        datetime.date(2020, 2, 1),
        datetime.timedelta(days=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 2, 1), 
        datetime.date(2020, 2, 15),
        datetime.timedelta(days=1),
    )
])

X1_train, X2_train, y_train = preprocess_c(df_train)
X1_val, X2_val, y_val = preprocess_c(df_val)

X1_train.shape, X2_train.shape, y_train.shape, X1_val.shape, X2_val.shape, y_val.shape

((1598125, 1, 3),
 (1598125, 14, 3),
 (1598125, 3),
 (263845, 1, 3),
 (263845, 14, 3),
 (263845, 3))

In [3]:
from keras.models import load_model, Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

In [4]:
# retrain view
model_view: Model = load_model('../model/c-view.keras')
cp_view = ModelCheckpoint('../model/c-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit([X1_train, X2_train], y_train[:, 0], validation_data=([X1_val, X2_val], y_val[:, 0]), epochs=10, callbacks=[cp_view])

Epoch 1/10
49942/49942 [==============================] - 85s 2ms/step - loss: 2.0695e-06 - root_mean_squared_error: 0.0014 - val_loss: 7.2352e-06 - val_root_mean_squared_error: 0.0027
Epoch 2/10
49942/49942 [==============================] - 93s 2ms/step - loss: 1.9794e-06 - root_mean_squared_error: 0.0014 - val_loss: 6.9384e-06 - val_root_mean_squared_error: 0.0026
Epoch 3/10
49942/49942 [==============================] - 85s 2ms/step - loss: 1.8781e-06 - root_mean_squared_error: 0.0014 - val_loss: 6.6649e-06 - val_root_mean_squared_error: 0.0026
Epoch 4/10
49942/49942 [==============================] - 85s 2ms/step - loss: 1.8366e-06 - root_mean_squared_error: 0.0014 - val_loss: 7.0932e-06 - val_root_mean_squared_error: 0.0027
Epoch 5/10
49942/49942 [==============================] - 87s 2ms/step - loss: 1.7848e-06 - root_mean_squared_error: 0.0013 - val_loss: 8.1543e-06 - val_root_mean_squared_error: 0.0029
Epoch 6/10
49942/49942 [==============================] - 88s 2ms/step - lo

In [5]:
# retrain cart
model_cart: Model = load_model('../model/c-cart.keras')
cp_cart = ModelCheckpoint('../model/c-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit([X1_train, X2_train], y_train[:, 1], validation_data=([X1_val, X2_val], y_val[:, 1]), epochs=10, callbacks=[cp_cart])

Epoch 1/10
49942/49942 [==============================] - 88s 2ms/step - loss: 9.9787e-06 - root_mean_squared_error: 0.0032 - val_loss: 1.3923e-05 - val_root_mean_squared_error: 0.0037
Epoch 2/10
49942/49942 [==============================] - 87s 2ms/step - loss: 9.8237e-06 - root_mean_squared_error: 0.0031 - val_loss: 1.1431e-05 - val_root_mean_squared_error: 0.0034
Epoch 3/10
49942/49942 [==============================] - 87s 2ms/step - loss: 9.6502e-06 - root_mean_squared_error: 0.0031 - val_loss: 1.0384e-05 - val_root_mean_squared_error: 0.0032
Epoch 4/10
49942/49942 [==============================] - 87s 2ms/step - loss: 9.4613e-06 - root_mean_squared_error: 0.0031 - val_loss: 1.1035e-05 - val_root_mean_squared_error: 0.0033
Epoch 5/10
49942/49942 [==============================] - 86s 2ms/step - loss: 9.4585e-06 - root_mean_squared_error: 0.0031 - val_loss: 1.0341e-05 - val_root_mean_squared_error: 0.0032
Epoch 6/10
49942/49942 [==============================] - 86s 2ms/step - lo

In [6]:
# retrain purchase
model_purchase: Model = load_model('../model/c-purchase.keras')
cp_purchase = ModelCheckpoint('../model/c-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit([X1_train, X2_train], y_train[:, 2], validation_data=([X1_val, X2_val], y_val[:, 2]), epochs=10, callbacks=[cp_purchase])

Epoch 1/10
49942/49942 [==============================] - 86s 2ms/step - loss: 3.7791e-05 - root_mean_squared_error: 0.0061 - val_loss: 2.5931e-05 - val_root_mean_squared_error: 0.0051
Epoch 2/10
49942/49942 [==============================] - 84s 2ms/step - loss: 3.7083e-05 - root_mean_squared_error: 0.0061 - val_loss: 2.5653e-05 - val_root_mean_squared_error: 0.0051
Epoch 3/10
49942/49942 [==============================] - 85s 2ms/step - loss: 3.6665e-05 - root_mean_squared_error: 0.0061 - val_loss: 3.5247e-05 - val_root_mean_squared_error: 0.0059
Epoch 4/10
49942/49942 [==============================] - 84s 2ms/step - loss: 3.6077e-05 - root_mean_squared_error: 0.0060 - val_loss: 2.6202e-05 - val_root_mean_squared_error: 0.0051
Epoch 5/10
49942/49942 [==============================] - 88s 2ms/step - loss: 3.5988e-05 - root_mean_squared_error: 0.0060 - val_loss: 2.5276e-05 - val_root_mean_squared_error: 0.0050
Epoch 6/10
49942/49942 [==============================] - 98s 2ms/step - lo